# Basic Fas Solver

This notebook follows the ``ranking_embedding`` one. 

We designed a simple minimum feedback arc set solver. In this notebook, we incorporate constraints into the solver. 

We later introduce a different solver, based on quick sorting and pivoting.

In [1]:
import sys
import numba
import numpy as np
sys.path.append('../..')
from ranking.fassolver.embedding import (
    fill_sym_emb, fill_emb_f8, canonical_map, 
    get_sym_emb, get_emb, get_emb_from_rank, fill_emb_from_rank
) 

In [2]:
np.random.seed(0)

Let's consider a feedback arc set problem, with target $$\inf <\phi(y), c>,$$ and constraint 
$$\phi(y)_{ij} = \text{const}_{ij}, \qquad\text{if}\qquad \text{const}_{ij} \neq 0.$$

In [3]:
m = 10
m_emb = (m*(m-1)) // 2
ind_map = canonical_map(m)
c = np.random.randn(m_emb)

In the following, we suggest to solve the unconstraint problem first, before incorporating the constraint

In [4]:
class BasicFasSolver:
    def __init__(self, ind_map):
        self.ind_map = ind_map
        
        # Placeholders
        m = len(ind_map)
        self.sym_pl = np.empty((m, m), dtype=np.float)
        self.score_pl = np.empty(m, dtype=np.float)
    
    def solve(self, c):
        """
        Solve inf_y <phi(y), c>.
        """
        emb = np.empty(c.shape, dtype=np.float)
        fill_sym_emb(c, self.sym_pl, self.ind_map)
        np.sum(self.sym_pl, axis=1, out=self.score_pl) 
        self.score_pl *= -1
        fill_emb_f8(self.score_pl, emb, self.ind_map)
        return emb
    
    def pre_solve(self, c):
        """
        Solve inf_y <phi(y), c>.
        """
        fill_sym_emb(c, self.sym_pl, self.ind_map)
        scores = np.sum(self.sym_pl, axis=1)
        scores *= -1
        rank = scores.argsort()
        return rank, scores

In [5]:
ind_map = canonical_map(m)
solver = BasicFasSolver(ind_map)
rank, scores = solver.pre_solve(c)
print(rank)

[0 1 9 7 3 6 8 2 4 5]


Let's get some constraint from partial ordering

In [6]:
sigma = np.random.permutation(m)
const = get_emb(sigma, ind_map)
const *= (np.random.randn(m_emb) > .5).astype(np.float)
const[const == 0] = 0
print(const)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0. -1.
  1.  1.  0.  0. -1.  0.  1.  0.  1.]


To incorporate those constraint, we perform a insertion-like sort. 

We build a list reviewing element from the smallest to the biggest. 

We insert element from the least to the most prefered ones based on the unconstraint solution. 

When inserting an element, one should first insert smaller element according to the constraints.

In [7]:
class BasicFasSolver:
    def __init__(self, ind_map):
        self.ind_map = ind_map
        
        # Placeholders
        m = len(ind_map)
        self.sym_pl = np.empty((m, m), dtype=np.float)
        self.score_pl = np.empty(m, dtype=np.float)
        self.const_pl = np.empty((m, m), dtype=np.bool_)
        self.range_pl = np.arange(m)
        self.rank_pl = np.empty(m, dtype=np.int)
        self.visited = np.empty(m, dtype=np.bool_)
    
    def solve(self, c):
        """
        Solve inf_y <phi(y), c>.
        """
        emb = np.empty(c.shape, dtype=np.float)
        self.solve_out(c, emb)
        return emb
    
    def solve_out(self, c, out):
        fill_sym_emb(c, self.sym_pl, self.ind_map)
        np.sum(self.sym_pl, axis=1, out=self.score_pl) 
        self.score_pl *= -1
        fill_emb_f8(self.score_pl, out, self.ind_map)
    
    def solve_const(self, c, const):
        pre_sol = self.pre_solve(c)
        return self.resolve_const(pre_sol, const)

    def pre_solve(self, c):
        fill_sym_emb(c, self.sym_pl, self.ind_map)
        scores = np.sum(self.sym_pl, axis=1)
        scores *= -1
        rank = scores.argsort()
        return rank, scores

    def resolve_const(self, pre_sol, const):
        emb = np.empty(c.shape, dtype=np.float)
        self.resolve_const_out(pre_sol, const, emb)
        return emb    

    def resolve_const_out(self, pre_sol, const, out):
        rank, scores = pre_sol

        fill_sym_emb(const, self.sym_pl, self.ind_map)
        np.equal(self.sym_pl, 1, out=self.const_pl)

        self.visited[:] = False
        for item in rank:
            self.insert_const(item, scores)
        fill_emb_from_rank(self.rank_pl, out, self.ind_map)
    
    def insert_const(self, item, scores):
        if self.visited[item]:
            return
        ind = self.const_pl[item] & (~self.visited)
        sm_items = self.range_pl[ind]
        sm_items = sm_items[scores[sm_items].argsort()]

        for i in sm_items:
            self.insert_const(i, scores)

        self.rank_pl[self.visited.sum()] = item
        self.visited[item] = True

In [8]:
solver = BasicFasSolver(ind_map)
sol_emb = solver.solve_const(c, const)
print(np.abs((sol_emb - const)[const != 0]).max())
print(((sol_emb - solver.solve(c)) == 0).mean())

0.0
0.7333333333333333


## Speeding up the code

In [9]:
@numba.jit('(i8, f8[::1], i8[::1], b1[::1], b1[:,::1], i8[::1])', nopython=True)
def _insert_const(item, scores, rank_pl, visited, const_pl, range_pl):
    if visited[item]:
        return
    ind = const_pl[item] & (~visited)
    sm_items = range_pl[ind]
    sm_items = sm_items[np.argsort(scores[sm_items])]

    for i in sm_items:
        _insert_const(i, scores, rank_pl, visited, const_pl, range_pl)

    rank_pl[visited.sum()] = item
    visited[item] = True

    
@numba.jit('(i8[::1], f8[::1], f8[::1], f8[::1], b1[::1], i8[::1],'\
            'f8[:,::1], b1[:,::1], i8[:,::1], i8[::1])', nopython=True)
def _resolve_out(rank, scores, const, out, visited, rank_pl, sym_pl, const_pl, ind_map, range_pl):
    fill_sym_emb(const, sym_pl, ind_map)
    for i in range_pl:
        for j in range_pl:
            const_pl[i,j] = sym_pl[i, j] == 1

    visited[:] = False
    for item in rank:
        _insert_const(item, scores, rank_pl, visited, const_pl, range_pl)
    fill_emb_from_rank(rank_pl, out, ind_map)

    
class BasicFasSolver:
    def __init__(self, ind_map):
        self.ind_map = ind_map
        
        # Placeholders
        m = len(ind_map)
        self.sym_pl = np.empty((m, m), dtype=np.float)
        self.score_pl = np.empty(m, dtype=np.float)
        self.const_pl = np.empty((m, m), dtype=np.bool_)
        self.range_pl = np.arange(m)
        self.rank_pl = np.empty(m, dtype=np.int)
        self.visited = np.empty(m, dtype=np.bool_)
    
    def solve(self, c):
        """
        Solve inf_y <phi(y), c>.
        """
        emb = np.empty(c.shape, dtype=np.float)
        self.solve_out(c, emb)
        return emb
    
    def solve_out(self, c, out):
        fill_sym_emb(c, self.sym_pl, self.ind_map)
        np.sum(self.sym_pl, axis=1, out=self.score_pl) 
        self.score_pl *= -1
        fill_emb_f8(self.score_pl, out, self.ind_map)
    
    def solve_const(self, c, const):
        pre_sol = self.pre_solve(c)
        return self.resolve_const(pre_sol, const)

    def pre_solve(self, c):
        fill_sym_emb(c, self.sym_pl, self.ind_map)
        scores = np.sum(self.sym_pl, axis=1)
        scores *= -1
        rank = scores.argsort()
        return rank, scores

    def resolve_const(self, pre_sol, const):
        emb = np.empty(c.shape, dtype=np.float)
        self.resolve_const_out(pre_sol, const, emb)
        return emb    

    def resolve_const_out(self, pre_sol, const, out):
        rank, scores = pre_sol
        _resolve_out(rank, scores, const, out, 
            self.visited, self.rank_pl, self.sym_pl, 
            self.const_pl, self.ind_map, self.range_pl)

In [10]:
fas_solver = BasicFasSolver(ind_map)
print(np.abs(sol_emb - fas_solver.solve_const(c, const)).max())
%timeit solver.solve_const(c, const)
%timeit fas_solver.solve_const(c, const)

0.0
171 µs ± 15.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
59.8 µs ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Define a FasSolver class for more lisibility

In [11]:
class FasSolver:
    def __init__(self, ind_map):
        self.ind_map = ind_map
        
        self.IL_met = '' # either pre_solve, either resolve
            
    def solve(self, c):
        """
        Solve the minimum feedback arc set problem reading:
           argmin_e <e, c>
        Subject to:
           - e Kendall's embedding of a permutation
        """
        emb = np.empty(c.shape, dtype=np.float)
        self.solve_out(c, emb)
        return emb
    
    def solve_const(self, c, const):
        """
        Solve the minimum feedback arc set problem reading:
           argmin_e <e, c>
        Subject to:
           - e[const != 0] = const
           - e Kendall's embedding of a permutation
        """
        emb = np.empty(c.shape, dtype=np.float)
        self.solve_const_out(c, const, emb)
        return emb
    
    def pre_solve(self, c):
        """
        First pass to solve the minimum feedback arc set problem reading:
           argmin_e <e, c>
        Subject to constraint not defined yet.
        
        This function allows to solve efficiently a big number of 
        instance with the same objective but different constraint.
        This is useful for the infimum loss.
        """
        raise NotImplementedError
        
    def incorporate_const(self, pre_sol, const):
        """
        Use pre solution of the problem argmin_e <e, c> and retune it to solve:
           argmin_e <e, c>
        Subject to:
           - e[const != 0] = const
           - e Kendall's embedding of a permutatiom

        This function allows to solve efficiently a big number of 
        instance with the same objective but different constraint.
        This is useful for the infimum loss.
        """
        emb = np.empty(c.shape, dtype=np.float)
        self.incorporate_const_out(pre_sol, const, emb)
        return emb

    def define_const(self, const):
        """
        Define constraint for the following ``resolve`` call.
        
        Useful for the disambiguation framework with warmstartable call.
        """
        raise NotImplementedError
        
    def resolve(self, c):
        """
        Solve by retaking last solution calculation:
           argmin_e <e, c>
        Subject to:
           - e[const != 0] = const
           - e Kendall's embedding of a permutation   
        where const has been define by ``define_const``.

        Useful for the disambiguation framework with warmstartable call.
        """
        emb = np.empty(c.shape, dtype=np.float)
        self.resolve_out(c, emb)
        return emb

    def solve_out(self, c, out):
        raise NotImplementedError

    def solve_const_out(self, c, const, out):
        raise NotImplementedError
        
#     def pre_solve(self, c):
#         raise NotImplementedError
        
    def incorporate_const_out(self, pre_sol, const, out):
        raise NotImplementedError

#     def define_const(self, const):
#         raise NotImplementedError

    def resolve_out(self, c, out):
        raise NotImplementedError

In [12]:
@numba.jit('(i8, f8[::1], i8[::1], b1[::1], b1[:,::1], i8[::1])', nopython=True)
def _insert_const(item, scores, rank_pl, visited, const_pl, range_pl):
    if visited[item]:
        return
    ind = const_pl[item] & (~visited)
    sm_items = range_pl[ind]
    sm_items = sm_items[np.argsort(scores[sm_items])]

    for i in sm_items:
        _insert_const(i, scores, rank_pl, visited, const_pl, range_pl)

    rank_pl[visited.sum()] = item
    visited[item] = True

    
@numba.jit('(i8[::1], f8[::1], f8[::1], f8[::1], b1[::1], i8[::1],'\
            'f8[:,::1], b1[:,::1], i8[:,::1], i8[::1])', nopython=True)
def _incorporate_const_out(rank, scores, const, out, visited, rank_pl, 
                           sym_pl, const_pl, ind_map, range_pl):
    fill_sym_emb(const, sym_pl, ind_map)
    for i in range_pl:
        for j in range_pl:
            const_pl[i,j] = sym_pl[i, j] == 1

    visited[:] = False
    for item in rank:
        _insert_const(item, scores, rank_pl, visited, const_pl, range_pl)
    fill_emb_from_rank(rank_pl, out, ind_map)

    
class BasicFasSolver(FasSolver):
    def __init__(self, ind_map):
        super(BasicFasSolver, self).__init__(ind_map)
        self.IL_met = 'pre_solve'
        
        # Placeholders
        m = len(ind_map)
        self.sym_pl = np.empty((m, m), dtype=np.float)
        self.score_pl = np.empty(m, dtype=np.float)
        self.const_pl = np.empty((m, m), dtype=np.bool_)
        self.range_pl = np.arange(m)
        self.rank_pl = np.empty(m, dtype=np.int)
        self.visited = np.empty(m, dtype=np.bool_)
   
    def solve_out(self, c, out):
        fill_sym_emb(c, self.sym_pl, self.ind_map)
        np.sum(self.sym_pl, axis=1, out=self.score_pl) 
        self.score_pl *= -1
        fill_emb_f8(self.score_pl, out, self.ind_map)

    def solve_const_out(self, c, const, out):
        pre_sol = self.pre_solve(c)
        self.incorporate_const_out(pre_sol, const, out)

    def pre_solve(self, c):
        """
        First pass to solve the minimum feedback arc set problem reading:
           argmin_e <e, c>
        Subject to constraint not defined yet.
        
        This function allows to solve efficiently a big number of 
        instance with the same objective but different constraint.
        This is useful for the infimum loss.
        """
        fill_sym_emb(c, self.sym_pl, self.ind_map)
        scores = np.sum(self.sym_pl, axis=1)
        scores *= -1
        rank = scores.argsort()
        return rank, scores
        
    def incorporate_const_out(self, pre_sol, const, out):
        rank, scores = pre_sol
        _incorporate_const_out(rank, scores, const, out, 
            self.visited, self.rank_pl, self.sym_pl, 
            self.const_pl, self.ind_map, self.range_pl)
        
#     def define_const(self, const):
#         raise NotImplementedError

#     def resolve_out(self, c, out):
#         raise NotImplementedError

In [13]:
fas_solver = BasicFasSolver(ind_map)
print(np.abs(sol_emb - fas_solver.solve_const(c, const)).max())
%timeit solver.solve_const(c, const)
%timeit fas_solver.solve_const(c, const)

0.0
242 µs ± 51 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
45.9 µs ± 3.77 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
